# Part 2. Solving Classification Problem

**Тренируем классификатор Logistic Regression для предсказания выживет ли пассажир**

In [1]:
# Импортируем необходимые библиотеки
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
# Загрузка датасета, подготовка признаков и целевой переменной согласно ДЗ №6
titanic = sns.load_dataset("titanic")
titanic_prepared = titanic.drop(['alive','class','embark_town','who','adult_male'], axis=1)
titanic_prepared['family'] = titanic_prepared['sibsp']+titanic_prepared['parch']
titanic_prepared = titanic_prepared.drop(['sibsp','parch','alone'], axis=1)
titanic_prepared['embarked'].fillna("S", inplace=True)
titanic_prepared['age'].fillna(titanic_prepared['age'].median(), inplace=True)
titanic_prepared = titanic_prepared.drop(['deck'], axis=1)
titanic_prepared = pd.get_dummies(titanic_prepared)
X = titanic_prepared.drop(columns=['survived'])
y = titanic_prepared['survived']

In [3]:
# Делим на выборки train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, )

In [4]:
# Используем нормализацию
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Обучение модели логистичесной регрессии
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [6]:
# Предсказания для train и test sets
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

**Рассчитаем эффективность Logistic Regression "вручную": выведим confusion matrix и вручную  посчитаем такие метрики, как accuracy, precision, recall и f1 (без готовых методов из библиотеки sklearn).**

In [7]:
# Реализация своих функции для ручных расчетов
def my_confusion_matrix(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    tn = sum((y_true == 0) & (y_pred == 0))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))

    return tp, fp, fn, tn

def my_accuracy(tp, fp, fn, tn):
    return (tp + tn) / (tp + tn + fp + fn)

def my_precision(tp, fp):
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def my_recall(tp, fn):
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def my_f1_score(precision, recall):
    return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

In [8]:
# Расчет метрик для train set
tp_train, fp_train, fn_train, tn_train = my_confusion_matrix(y_train, y_train_pred)
accuracy_train = my_accuracy(tp_train, fp_train, fn_train, tn_train)
precision_train = my_precision(tp_train, fp_train)
recall_train = my_recall(tp_train, fn_train)
f1_train = my_f1_score(precision_train, recall_train)

# Расчет метрик для test set
tp_test, fp_test, fn_test, tn_test = my_confusion_matrix(y_test, y_test_pred)
accuracy_test = my_accuracy(tp_test, fp_test, fn_test, tn_test)
precision_test = my_precision(tp_test, fp_test)
recall_test = my_recall(tp_test, fn_test)
f1_test = my_f1_score(precision_test, recall_test)

# Вывод результатов
print("Confusion Matrix для train set:")
print(tn_train, fp_train)
print(fn_train, tp_train)

print(f"Accuracy для train set: {accuracy_train:.2%}")
print(f"Precision для train set: {precision_train:.2%}")
print(f"Recall для train set: {recall_train:.2%}")
print(f"F1 Score для train set: {f1_train:.2%}")

print("\nConfusion Matrix для test set:")
print(tn_test, fp_test)
print(fn_test, tp_test)

print(f"Accuracy для test set: {accuracy_test:.2%}")
print(f"Precision для test set: {precision_test:.2%}")
print(f"Recall для test set: {recall_test:.2%}")
print(f"F1 Score для test set: {f1_test:.2%}")


Confusion Matrix для train set:
381 63
82 186
Accuracy для train set: 79.63%
Precision для train set: 74.70%
Recall для train set: 69.40%
F1 Score для train set: 71.95%

Confusion Matrix для test set:
90 15
20 54
Accuracy для test set: 80.45%
Precision для test set: 78.26%
Recall для test set: 72.97%
F1 Score для test set: 75.52%


**Теперь пересчитаем метрики accuracy, precision, recall и f1 с помощью готовых методов из sklearn.**

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Confusion Matrix и метрики для train set
print("Confusion Matrix для train set:")
print(confusion_matrix(y_train, y_train_pred))
print(f"Accuracy для train set: {accuracy_score(y_train, y_train_pred):.2%}")
print(f"Precision для train set: {precision_score(y_train, y_train_pred):.2%}")
print(f"Recall для train set: {recall_score(y_train, y_train_pred):.2%}")
print(f"F1 Score для train set: {f1_score(y_train, y_train_pred):.2%}")

# Confusion Matrix и метрики для test set
print("\nConfusion Matrix для test set:")
print(confusion_matrix(y_test, y_test_pred))
print(f"Accuracy для test set: {accuracy_score(y_test, y_test_pred):.2%}")
print(f"Precision для test set: {precision_score(y_test, y_test_pred):.2%}")
print(f"Recall для test set: {recall_score(y_test, y_test_pred):.2%}")
print(f"F1 Score для test set: {f1_score(y_test, y_test_pred):.2%}")

Confusion Matrix для train set:
[[381  63]
 [ 82 186]]
Accuracy для train set: 79.63%
Precision для train set: 74.70%
Recall для train set: 69.40%
F1 Score для train set: 71.95%

Confusion Matrix для test set:
[[90 15]
 [20 54]]
Accuracy для test set: 80.45%
Precision для test set: 78.26%
Recall для test set: 72.97%
F1 Score для test set: 75.52%


**Обучим на данных другие модели  - SVM, Decision Tree, Random Forest, для сравнения. Посчитаем train/test метрики для данных моделей.**

In [10]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Инициализация моделей
models = {
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier()
}

# Обучение и оценка моделей
for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    print(f"\nМетрики для {name}:")
    # Confusion Matrix и метрики для train set
    print("\nConfusion Matrix для train set:")
    print(confusion_matrix(y_train, y_train_pred))
    print(f"Accuracy для train set: {accuracy_score(y_train, y_train_pred):.2%}")
    print(f"Precision для train set: {precision_score(y_train, y_train_pred):.2%}")
    print(f"Recall для train set: {recall_score(y_train, y_train_pred):.2%}")
    print(f"F1 Score для train set: {f1_score(y_train, y_train_pred):.2%}")

    # Confusion Matrix и метрики для test set
    print("\nConfusion Matrix для test set:")
    print(confusion_matrix(y_test, y_test_pred))
    print(f"Accuracy для test set: {accuracy_score(y_test, y_test_pred):.2%}")
    print(f"Precision для test set: {precision_score(y_test, y_test_pred):.2%}")
    print(f"Recall для test set: {recall_score(y_test, y_test_pred):.2%}")
    print(f"F1 Score для test set: {f1_score(y_test, y_test_pred):.2%}")



Метрики для SVM:

Confusion Matrix для train set:
[[417  27]
 [201  67]]
Accuracy для train set: 67.98%
Precision для train set: 71.28%
Recall для train set: 25.00%
F1 Score для train set: 37.02%

Confusion Matrix для test set:
[[99  6]
 [55 19]]
Accuracy для test set: 65.92%
Precision для test set: 76.00%
Recall для test set: 25.68%
F1 Score для test set: 38.38%

Метрики для Decision Tree:

Confusion Matrix для train set:
[[443   1]
 [ 14 254]]
Accuracy для train set: 97.89%
Precision для train set: 99.61%
Recall для train set: 94.78%
F1 Score для train set: 97.13%

Confusion Matrix для test set:
[[81 24]
 [18 56]]
Accuracy для test set: 76.54%
Precision для test set: 70.00%
Recall для test set: 75.68%
F1 Score для test set: 72.73%

Метрики для Random Forest:

Confusion Matrix для train set:
[[439   5]
 [ 10 258]]
Accuracy для train set: 97.89%
Precision для train set: 98.10%
Recall для train set: 96.27%
F1 Score для train set: 97.18%

Confusion Matrix для test set:
[[90 15]
 [16 58]

**Выберем гиперпараметры для лучшей модели с помощью Random Search.**

In [11]:
rf_model = RandomForestClassifier()
from sklearn.model_selection import RandomizedSearchCV
# Создадим множество гиперпараметров, из которых выберется лучшая комбинация
params = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': [None] + list(np.arange(10, 50, 5)),
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Производим поиск Random Search
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=params, n_iter=20, cv=5, scoring='f1_macro', random_state=42)
random_search.fit(X_train, y_train)
# Вывод результатов
print("\nRandomized Search Best Parameters:")
print(random_search.best_params_)
print("Randomized Search Best Score:", random_search.best_score_)



Randomized Search Best Parameters:
{'n_estimators': 140, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40}
Randomized Search Best Score: 0.8141413591107973


**Перетренируем лучшую модель с лучшими гиперпараметрами. Сделаем замеры по метрикам.**

In [12]:
#Перетренируем модель с лучшими гиперпараметрами
rf_model = RandomForestClassifier(n_estimators=140, min_samples_split=15, min_samples_leaf=1, max_features='sqrt', max_depth=40)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Confusion Matrix и метрики для train set
print("Confusion Matrix для train set:")
print(confusion_matrix(y_train, y_train_pred))
print(f"Accuracy для train set: {accuracy_score(y_train, y_train_pred):.2%}")
print(f"Precision для train set: {precision_score(y_train, y_train_pred):.2%}")
print(f"Recall для train set: {recall_score(y_train, y_train_pred):.2%}")
print(f"F1 Score для train set: {f1_score(y_train, y_train_pred):.2%}")

# Confusion Matrix и метрики для test set
print("\nConfusion Matrix для test set:")
print(confusion_matrix(y_test, y_test_pred))
print(f"Accuracy для test set: {accuracy_score(y_test, y_test_pred):.2%}")
print(f"Precision для test set: {precision_score(y_test, y_test_pred):.2%}")
print(f"Recall для test set: {recall_score(y_test, y_test_pred):.2%}")
print(f"F1 Score для test set: {f1_score(y_test, y_test_pred):.2%}")

Confusion Matrix для train set:
[[439   5]
 [ 10 258]]
Accuracy для train set: 97.89%
Precision для train set: 98.10%
Recall для train set: 96.27%
F1 Score для train set: 97.18%

Confusion Matrix для test set:
[[89 16]
 [16 58]]
Accuracy для test set: 82.12%
Precision для test set: 78.38%
Recall для test set: 78.38%
F1 Score для test set: 78.38%


**Выводы и наблюдения**

После применения разных методов классификации, я заметил, что SVM оказался менее эффективным, в то время как логистическая регрессия, Decision Tree и Random Forest показали похожую производительность на тестовом наборе данных.

Похоже, что SVM показал менее эффективную производительность из-за того, что не было подбора параметров или предварительной обработки данных конкретно для модели SVM.

Decision Tree и Random Forest показали более высокую производительность на тренировочном наборе данных из-за своей способности к "запоминанию" деталей тренировочных данных. Decision Tree может создавать глубокие ветви, чтобы точно соответствовать обучающим данным, а Random Forest, как ансамблевый метод, комбинирует несколько деревьев, улучшая обобщающую способность модели. Однако, это также может привести к переобучению на тренировочных данных, что привело к ухудшению производительности на тестовом наборе данных.

Когда я использовал Random Search для подбора гиперпараметров для Random Forest, я получил примерно такие же результаты, что и без этой настройки. Это может быть потому, что Random Search выбирает параметры наугад из заданных диапазонов, и некоторые комбинации параметров могут оказаться не самыми оптимальными для наших данных. Возможно, идеальные параметры лежат за пределами того, где мы искали.

В целом, Random Forest выглядит наиболее предпочтительным в нашем случае, показывая немного высокую производительность чем остальные модели как на обучающем, так и на тестовом наборе данных.